# Natural Language to SQL Query System


This project converts **natural language questions** (in English) into **SQL queries**, executes them on a sample SQLite Employee Database, and returns the results in real time.  
It uses a **T5 model fine-tuned on WikiSQL** and a simple **Gradio interface** for interaction.


In [1]:
# Cell 1 - Install dependencies
!pip install transformers gradio datasets


In [2]:
# Cell 2 - Import libraries
import sqlite3
import re
import gradio as gr
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


In [3]:
# Cell 3 - Create Example Employee Database (so it runs everywhere)
conn = sqlite3.connect("employee.db")
cursor = conn.cursor()

cursor.execute("DROP TABLE IF EXISTS Employee;")
cursor.execute("""
CREATE TABLE Employee (
    ID INTEGER PRIMARY KEY,
    Name TEXT,
    Country TEXT,
    Department TEXT,
    Salary INTEGER
);
""")

cursor.executemany("""
INSERT INTO Employee (Name, Country, Department, Salary) VALUES (?, ?, ?, ?)
""", [
    ("Alice", "USA", "HR", 60000),
    ("Bob", "Bangladesh", "IT", 55000),
    ("Charlie", "India", "Finance", 70000),
    ("Diana", "USA", "IT", 80000),
    ("Esha", "Bangladesh", "Finance", 65000),
])

conn.commit()
conn.close()


In [4]:
# Cell 4 - Load Pretrained Model (T5 fine-tuned on WikiSQL)
model_name = "mrm8488/t5-base-finetuned-wikiSQL"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

In [5]:
# Cell 5 - Connect to SQLite Database (fresh connection per query)
def get_connection():
    return sqlite3.connect("employee.db")


In [6]:
# Cell 6 - Function: Natural Language → SQL → Execute → Results
def query_employee_db(question):
    try:
        # 1. Generate SQL from natural language
        input_text = "translate to SQL: " + question
        inputs = tokenizer(input_text, return_tensors="pt", max_length=128, truncation=True)
        outputs = model.generate(**inputs, max_length=128)
        generated_sql = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # 2. Cleanup SQL
        generated_sql = generated_sql.replace("table", "Employee")
        generated_sql = generated_sql.replace("Employees", "*")
        generated_sql = re.sub(r"= ([A-Za-z]+)", r"= '\1'", generated_sql)

        # 3. Execute SQL
        conn = get_connection()
        cursor = conn.cursor()
        cursor.execute(generated_sql)
        rows = cursor.fetchall()
        conn.close()

        # 4. Return results
        if rows:
            return f"🔎 SQL: {generated_sql}\n\n✅ Results:\n" + "\n".join(map(str, rows))
        else:
            return f"🔎 SQL: {generated_sql}\n\n⚠️ No results found."

    except Exception as e:
        return f"❌ Error: {str(e)}"


In [9]:
# ✅ Corrected Cell 7 - Launch Gradio Interface with table output

import pandas as pd

def query_employee_db(question):
    try:
        # 1. Generate SQL from natural language
        input_text = "translate to SQL: " + question
        inputs = tokenizer(input_text, return_tensors="pt", max_length=128, truncation=True)
        outputs = model.generate(**inputs, max_length=128)
        generated_sql = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # 2. Basic SQL cleanup
        generated_sql = generated_sql.replace("table", "Employee")
        generated_sql = generated_sql.replace("Employees", "*")
        generated_sql = re.sub(r"= ([A-Za-z]+)", r"= '\1'", generated_sql)

        # 3. Execute SQL on database
        conn = sqlite3.connect("employee.db")  # ✅ reconnect inside function (thread-safe)
        cursor = conn.cursor()
        cursor.execute(generated_sql)
        rows = cursor.fetchall()
        col_names = [desc[0] for desc in cursor.description] if cursor.description else []
        conn.close()

        # 4. Format results for Gradio
        if rows:
            df = pd.DataFrame(rows, columns=col_names)
            return generated_sql, df
        else:
            return generated_sql, pd.DataFrame([["No results found"]], columns=["Message"])

    except Exception as e:
        return "Error", pd.DataFrame([[str(e)]], columns=["Message"])


# ✅ Gradio with two outputs: SQL + Table
demo = gr.Interface(
    fn=query_employee_db,
    inputs=gr.Textbox(label="Ask in English", placeholder="e.g., Show all employees from Bangladesh"),
    outputs=[
        gr.Textbox(label="Generated SQL"),
        gr.Dataframe(label="Query Results", wrap=True)
    ],
    title="Natural Language → SQL → Employee DB",
    description="Ask questions in English. The model converts them to SQL, runs them on the Employee DB, and shows results."
)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6ca72e66157f393910.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
